# [Torch Script](https://tutorials.pytorch.kr/beginner/Intro_to_TorchScript_tutorial.html)

- 기존 모듈 추적
- 스크립트를 사용하여 모듈을 직접 컴파일
- 두 가지 접근 방식을 구성하는 방법
- TorchScript 모듈 저장 및 로드

In [3]:
import torch
print(torch.__version__)

1.2.0


## Pytorch 모델 작성의 기초
- Module은 Pytorch의 기본 구성 단위
    1. 호출을 위해 모듈을 준비하는 생성자
    2. `Parameters` 및 하위 집합 `Modules`
        - 생성자에 의해 초기화되며 호출 중에 모듈에 의해 사용될 수 있음
    3. `forward`기능
        - 모듈이 호출 될 때 실행되는 코드

In [6]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
    # 모듈이 호출 될 때 실행
    def forward(self, x, h):
        new_h = torch.tanh(x + h)
        return new_h, new_h
    
my_cell = MyCell()
x = torch.rand(3, 4)
h = torch.rand(3, 4)
print(my_cell(x, h)[0])
print(my_cell(x, h)[1])

tensor([[0.5601, 0.2557, 0.7294, 0.1307],
        [0.7899, 0.6791, 0.8398, 0.7546],
        [0.5563, 0.6946, 0.7326, 0.4131]])
tensor([[0.5601, 0.2557, 0.7294, 0.1307],
        [0.7899, 0.6791, 0.8398, 0.7546],
        [0.5563, 0.6946, 0.7326, 0.4131]])


__위에서__
1. 하위 클래스를 만드는 클래스를 만듬
    - `torch.nn.Module`
2. 생성자를 정의
    - `super(class, self).__init__()
3. `forwad` 두 개의 입력을 받아 두 개의 출력을 반환하는 함수 정의

- 모델을 인스턴스화하고, x, h 를 (3,4)의 임의의값으로 셀 호출 => forward 

In [13]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h, new_h

my_cell = MyCell()
# 프린트 출력으로 Module을 시각적으로 볼 수 있음
print(my_cell)
print(my_cell(x, h))

MyCell(
  (linear): Linear(in_features=4, out_features=4, bias=True)
)
(tensor([[ 0.3063, -0.3306,  0.6263,  0.2166],
        [ 0.5449,  0.0184,  0.7154,  0.5830],
        [ 0.2641,  0.0472,  0.5797,  0.2279]], grad_fn=<TanhBackward>), tensor([[ 0.3063, -0.3306,  0.6263,  0.2166],
        [ 0.5449,  0.0184,  0.7154,  0.5830],
        [ 0.2641,  0.0472,  0.5797,  0.2279]], grad_fn=<TanhBackward>))


__위에서__

- `self.linear` 속성 추가하고 `forwad`함수 호출
- 계층구조를 구축한 Module
- print 출력문으로 Module을 시각화할 수 있음

In [16]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        # 호출될 때 forward 실행
        self.dg = MyDecisionGate()
        self.linear = torch.nn.Linear(4, 4)
        
    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h

my_cell = MyCell()
print(my_cell)
print(my_cell(x, h))

MyCell(
  (dg): MyDecisionGate()
  (linear): Linear(in_features=4, out_features=4, bias=True)
)
(tensor([[0.1294, 0.5614, 0.7005, 0.2993],
        [0.6078, 0.8134, 0.7728, 0.7225],
        [0.3048, 0.7784, 0.6045, 0.5626]], grad_fn=<TanhBackward>), tensor([[0.1294, 0.5614, 0.7005, 0.2993],
        [0.6078, 0.8134, 0.7728, 0.7225],
        [0.3048, 0.7784, 0.6045, 0.5626]], grad_fn=<TanhBackward>))


__위에서__

- `MyDecisionGate`를 통해 제어흐름을 사용
- 많은 프레임워크는 완전한 프로그램 표현에 대한 상징적 파생물을 계산하는 접근법이지만, `Pytorch`에서는 `Gradient-tape`를 사용
    - 정방향 단계로 수행하는 모든 연산 기억하고, 역방향 단계로 거꾸로 연산들을 재생(autograd 작동 방식)

## TorchScript의 기초
- TorchScript는 Pytorch의 유연하고 동적인 특성을 고려하여 모델의 정의를 캡처하는 도구를 제공

### Tracing Modules

In [18]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)
        
    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h, new_h
    
my_cell = MyCell()
x, h = torch.rand(3, 4), torch.rand(3, 4)
# TraceModule 인스턴스
traced_cell = torch.jit.trace(my_cell, (x, h))
print(traced_cell)
traced_cell(x,h)

TracedModule[MyCell](
  (linear): TracedModule[Linear]()
)


(tensor([[ 0.6862,  0.3827, -0.3673,  0.5626],
         [ 0.4641,  0.1894, -0.0544,  0.5898],
         [ 0.9317, -0.3142, -0.6642,  0.8332]],
        grad_fn=<DifferentiableGraphBackward>),
 tensor([[ 0.6862,  0.3827, -0.3673,  0.5626],
         [ 0.4641,  0.1894, -0.0544,  0.5898],
         [ 0.9317, -0.3142, -0.6642,  0.8332]],
        grad_fn=<DifferentiableGraphBackward>))

- TorchScript는 일반적으로 딥러닝에서 그래프라고하는 중간표현(또는 IR)에 정의를 기록
- .graph속성을 사용하여 그래프를 검사할 수 있음

In [21]:
type(traced_cell)

torch.jit.TopLevelTracedModule

In [19]:
# 저 수준의 정보
print(traced_cell.graph)

graph(%self : ClassType<MyCell>,
      %input : Float(3, 4),
      %h : Float(3, 4)):
  %1 : ClassType<Linear> = prim::GetAttr[name="linear"](%self)
  %weight : Tensor = prim::GetAttr[name="weight"](%1)
  %bias : Tensor = prim::GetAttr[name="bias"](%1)
  %6 : Float(4!, 4!) = aten::t(%weight), scope: MyCell/Linear[linear] # C:\ProgramData\Anaconda3\envs\pytch_env\lib\site-packages\torch\nn\functional.py:1369:0
  %7 : int = prim::Constant[value=1](), scope: MyCell/Linear[linear] # C:\ProgramData\Anaconda3\envs\pytch_env\lib\site-packages\torch\nn\functional.py:1369:0
  %8 : int = prim::Constant[value=1](), scope: MyCell/Linear[linear] # C:\ProgramData\Anaconda3\envs\pytch_env\lib\site-packages\torch\nn\functional.py:1369:0
  %9 : Float(3, 4) = aten::addmm(%bias, %input, %6, %7, %8), scope: MyCell/Linear[linear] # C:\ProgramData\Anaconda3\envs\pytch_env\lib\site-packages\torch\nn\functional.py:1369:0
  %10 : int = prim::Constant[value=1](), scope: MyCell # <ipython-input-18-2798a6abece9>:

In [20]:
print(traced_cell.code)

def forward(self,
    input: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  _0 = self.linear
  weight = _0.weight
  bias = _0.bias
  _1 = torch.addmm(bias, input, torch.t(weight), beta=1, alpha=1)
  _2 = torch.tanh(torch.add(_1, h, alpha=1))
  return (_2, _2)



- GIL(Global Interpreter Lock) : 컴퓨터 언어 인터프리터에서 스레드 실행을 동기화하여 한 번에 하나의 기본 스레드만 실행할 수 있도록하는 메커니즘
    - 멀티 코어 프로세스에서 실행되더라도 한 번에 하나의 스레드만 실행

### TorchScript를 사용하는 이유
1. TorchScript코드는 기본적으로 제한된 Python 자체 인터프리터에서 호출할 수 있음
    - 이 인터프리터는 GIL을 얻지않으므로 동일한 인스턴스에서 동시에 많은 요청을 처리할 수 있음
2. 이 형식을 사용하면 전체 모델을 디스크에 저장하고 Python 이외의 언어로 작성된 서버와 같은 다른 환경에로드할 수 있음
3. TorchScript는 코드에서 컴파일러 최적화를 수행하여 보다 효율적인 실행을 제공할 수 있는 표현을 제공
4. TorchScript를 사용하면 개별 연산자보다 프로그램을 더 넓게 볼 수 있는 많은 백엔드/장치 런타임과 인터페이스할 수 있음

In [22]:
# 동일한 결과
print(my_cell(x, h))
print(traced_cell(x, h))

(tensor([[ 0.6862,  0.3827, -0.3673,  0.5626],
        [ 0.4641,  0.1894, -0.0544,  0.5898],
        [ 0.9317, -0.3142, -0.6642,  0.8332]], grad_fn=<TanhBackward>), tensor([[ 0.6862,  0.3827, -0.3673,  0.5626],
        [ 0.4641,  0.1894, -0.0544,  0.5898],
        [ 0.9317, -0.3142, -0.6642,  0.8332]], grad_fn=<TanhBackward>))
(tensor([[ 0.6862,  0.3827, -0.3673,  0.5626],
        [ 0.4641,  0.1894, -0.0544,  0.5898],
        [ 0.9317, -0.3142, -0.6642,  0.8332]],
       grad_fn=<DifferentiableGraphBackward>), tensor([[ 0.6862,  0.3827, -0.3673,  0.5626],
        [ 0.4641,  0.1894, -0.0544,  0.5898],
        [ 0.9317, -0.3142, -0.6642,  0.8332]],
       grad_fn=<DifferentiableGraphBackward>))


## 스크립트를 사용하여 모듈 변환

In [24]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x
        
class MyCell(torch.nn.Module):
    def __init__(self, dg):
        super(MyCell, self).__init__()
        self.dg = dg
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h
    
my_cell = MyCell(MyDecisionGate())
traced_cell = torch.jit.trace(my_cell, (x, h))
print(traced_cell.code)

def forward(self,
    input: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  _0 = self.linear
  weight = _0.weight
  bias = _0.bias
  x = torch.addmm(bias, input, torch.t(weight), beta=1, alpha=1)
  _1 = torch.tanh(torch.add(x, h, alpha=1))
  return (_1, _1)



C:\ProgramData\Anaconda3\envs\pytch_env\lib\site-packages\ipykernel_launcher.py:3: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  This is separate from the ipykernel package so we can avoid doing imports until


__위에서__

- `.code`출력을 보면 `if-else`분기가 어디에도 없는 것을 알 수 있음
- 추적 코드는 발생하는 작업을 기록하기 때문에 정확하게 수행하는 ScriptModule을 구성하는 것과 동일하게 수행
    - 제어 흐름과 같은 것은 지워짐
   
- __하지만__Python 소스 코드를 직접 분석하여 TorchScript로 변환하는 스크립트 컴파일러 제공

In [30]:
# 스크립트 컴파일 통해 MyDecisionGate 변환
scripted_gate = torch.jit.script(MyDecisionGate())

my_cell = MyCell(scripted_gate)
traced_cell = torch.jit.script(my_cell)
print(traced_cell.code)

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  _0 = self.linear
  _1 = _0.weight
  _2 = _0.bias
  if torch.eq(torch.dim(x), 2):
    _3 = torch.__isnot__(_2, None)
  else:
    _3 = False
  if _3:
    bias = ops.prim.unchecked_unwrap_optional(_2)
    ret = torch.addmm(bias, x, torch.t(_1), beta=1, alpha=1)
  else:
    output = torch.matmul(x, torch.t(_1))
    if torch.__isnot__(_2, None):
      bias0 = ops.prim.unchecked_unwrap_optional(_2)
      output0 = torch.add_(output, bias0, alpha=1)
    else:
      output0 = output
    ret = output0
  _4 = torch.gt(torch.sum(ret, dtype=None), 0)
  if bool(_4):
    _5 = ret
  else:
    _5 = torch.neg(ret)
  new_h = torch.tanh(torch.add(_5, h, alpha=1))
  return (new_h, new_h)



In [31]:
x, h = torch.rand(3, 4), torch.rand(3, 4)
traced_cell(x, h)

(tensor([[0.4201, 0.8930, 0.1284, 0.9095],
         [0.4096, 0.7360, 0.6275, 0.6760],
         [0.2539, 0.6246, 0.1710, 0.6778]],
        grad_fn=<DifferentiableGraphBackward>),
 tensor([[0.4201, 0.8930, 0.1284, 0.9095],
         [0.4096, 0.7360, 0.6275, 0.6760],
         [0.2539, 0.6246, 0.1710, 0.6778]],
        grad_fn=<DifferentiableGraphBackward>))

In [36]:
x

tensor([[0.7090, 0.0531, 0.7196, 0.8786],
        [0.8287, 0.5535, 0.2831, 0.4030],
        [0.4868, 0.3671, 0.2124, 0.9849]])

## 스크립팅 및 추적 혼합
- 어떤 상황에서는 스크립팅 보다 추적을 사용해야함
    - 모듈에는 TorchScript에 표시하지 않으려는 상수 Python값을 기반으로 결정되는 아키텍쳐가 있음
    - 이 경우 스크립팅은 추적으로 구성될 수 있음

In [38]:
class MyRNNLoop(torch.nn.Module):
    def __init__(self):
        super(MyRNNLoop, self).__init__()
        self.cell = torch.jit.trace(MyCell(scripted_gate), (x, h))
    
    def forward(self, xs):
        h, y = torch.zeros(3, 4), torch.zeros(3, 4)
        for i in range(xs.size(0)):
            y, h = self.cell(xs[i], h)
        return y, h
    
rnn_loop = torch.jit.script(MyRNNLoop())
print(rnn_loop.code)

def forward(self,
    xs: Tensor) -> Tuple[Tensor, Tensor]:
  h = torch.zeros([3, 4], dtype=None, layout=None, device=None, pin_memory=None)
  y = torch.zeros([3, 4], dtype=None, layout=None, device=None, pin_memory=None)
  y0, h0 = y, h
  for i in range(torch.size(xs, 0)):
    _0 = self.cell
    _1 = torch.select(xs, 0, i)
    _2 = _0.linear
    weight = _2.weight
    bias = _2.bias
    _3 = torch.addmm(bias, _1, torch.t(weight), beta=1, alpha=1)
    _4 = torch.gt(torch.sum(_3, dtype=None), 0)
    if bool(_4):
      _5 = _3
    else:
      _5 = torch.neg(_3)
    _6 = torch.tanh(torch.add(_5, h0, alpha=1))
    y0, h0 = _6, _6
  return (y0, h0)



In [39]:
class WrapRNN(torch.nn.Module):
  def __init__(self):
    super(WrapRNN, self).__init__()
    self.loop = torch.jit.script(MyRNNLoop())

  def forward(self, xs):
    y, h = self.loop(xs)
    return torch.relu(y)

traced = torch.jit.trace(WrapRNN(), (torch.rand(10, 3, 4)))
print(traced.code)

def forward(self,
    argument_1: Tensor) -> Tensor:
  _0 = self.loop
  h = torch.zeros([3, 4], dtype=None, layout=None, device=None, pin_memory=None)
  h0 = h
  for i in range(torch.size(argument_1, 0)):
    _1 = _0.cell
    _2 = torch.select(argument_1, 0, i)
    _3 = _1.linear
    weight = _3.weight
    bias = _3.bias
    _4 = torch.addmm(bias, _2, torch.t(weight), beta=1, alpha=1)
    _5 = torch.gt(torch.sum(_4, dtype=None), 0)
    if bool(_5):
      _6 = _4
    else:
      _6 = torch.neg(_4)
    h0 = torch.tanh(torch.add(_6, h0, alpha=1))
  return torch.relu(h0)



## 모델 저장 및 로드
- 아카이브형식으로 디스크에 저장하고 로드하는 API 제공
    - 이 형식에는 코드, 매개 변수, 속성 및 디버그 정보가 포함됨
    - 아카이브는 완전히 별개의 프로세스로 로드할 수 있는 모델의 독립형 표현

In [41]:
traced.save('wrapped_rnn.zip')

loaded = torch.jit.load('wrapped_rnn.zip')

print(loaded)
print(loaded.code)

ScriptModule(
  (loop): ScriptModule(
    (cell): ScriptModule(
      (dg): ScriptModule()
      (linear): ScriptModule()
    )
  )
)
def forward(self,
    argument_1: Tensor) -> Tensor:
  _0 = self.loop
  h = torch.zeros([3, 4], dtype=None, layout=None, device=None, pin_memory=None)
  h0 = h
  for i in range(torch.size(argument_1, 0)):
    _1 = _0.cell
    _2 = torch.select(argument_1, 0, i)
    _3 = _1.linear
    weight = _3.weight
    bias = _3.bias
    _4 = torch.addmm(bias, _2, torch.t(weight), beta=1, alpha=1)
    _5 = torch.gt(torch.sum(_4, dtype=None), 0)
    if bool(_5):
      _6 = _4
    else:
      _6 = torch.neg(_4)
    h0 = torch.tanh(torch.add(_6, h0, alpha=1))
  return torch.relu(h0)

